In [1]:
import numpy as np
import pandas as pd

In [201]:
scaled = pd.read_csv('../dataset/scaled_nav.csv')
strategy = pd.read_excel('../dataset/strategy.xlsx')

In [206]:
a= pd.merge(scaled,strategy,on='fund_id',how = 'left')

In [210]:
a.drop(['Unnamed: 0','date_issuing','file_num'],axis=1,inplace=True)

In [211]:
a.to_csv('../dataset/scaled_strategy.csv')

# 交易记录数据

In [149]:
file_list = ['201904.csv','201905.csv','201906.csv',
            '201907.csv','201908.csv','201909.csv',
            '201910.csv','201911.csv','201912.csv',
            '202001.csv','202002.csv','202003.csv',
            '202004.csv','202005.csv','202006.csv']

# 原数据拼接和处理

In [350]:
data = pd.read_csv('../交易记录/'+file_list[0],error_bad_lines=False)
for i in range(1,len(file_list)):
    tmp = pd.read_csv('../交易记录/'+file_list[i],error_bad_lines=False)
    data = pd.concat([data,tmp],ignore_index=True)

b'Skipping line 56: expected 16 fields, saw 18\n'
b'Skipping line 91: expected 16 fields, saw 18\nSkipping line 92: expected 16 fields, saw 18\nSkipping line 114: expected 16 fields, saw 18\n'
b'Skipping line 9: expected 16 fields, saw 18\nSkipping line 19: expected 16 fields, saw 18\n'
b'Skipping line 22: expected 16 fields, saw 18\nSkipping line 43: expected 16 fields, saw 18\nSkipping line 44: expected 16 fields, saw 18\n'
b'Skipping line 12: expected 16 fields, saw 18\nSkipping line 70: expected 16 fields, saw 18\nSkipping line 83: expected 16 fields, saw 18\n'
b'Skipping line 18: expected 16 fields, saw 18\nSkipping line 36: expected 16 fields, saw 18\nSkipping line 41: expected 16 fields, saw 18\nSkipping line 55: expected 16 fields, saw 18\nSkipping line 82: expected 16 fields, saw 18\n'
b'Skipping line 136: expected 16 fields, saw 18\n'
b'Skipping line 148: expected 16 fields, saw 18\nSkipping line 186: expected 16 fields, saw 18\n'
b'Skipping line 23: expected 16 fields, saw 1

In [351]:
data['日期'] = pd.to_datetime(data['日期'],format = '%Y%m%d')

In [352]:
data['证券代码'] = data['证券代码'].apply(str)
ls = list(data['证券代码'])
for i in range(len(ls)):
    if ls[i] !='nan':
        ls[i] = ls[i][:-2]
for i in range(len(ls)):
    if len(ls[i])==4:
        ls[i] ='00'+ls[i]
    if len(ls[i])==5:
        ls[i]='0'+ls[i]
data['证券代码'] = pd.Series(ls)

In [353]:
set(list(data['操作']))

{'债券兑付',
 '兑息扣税',
 '利息归本',
 '基金资金拨入',
 '基金资金拨出',
 '市值申购中签',
 '托管转出',
 '担保品划出',
 '新股申购确认',
 '申购配号',
 '股息入账',
 '证券买入',
 '证券卖出',
 '转存管转入',
 '转存管转出',
 '银行转取',
 '银行转存'}

In [354]:
del_list = ['担保品划出','转存管转出','转存管转入','市值申购中签','托管转出','申购配号']

In [355]:
for i in del_list:
    data.drop(index=(data.loc[data['操作']==i].index),inplace=True)

In [356]:
len(data)

1165

In [357]:
data=data.reset_index()

In [347]:
data.tail()

,index,日期,流水号,证券代码,证券名称,操作,发生金额,后资金额,摘要,成交数量,成交均价,成交金额,交易市场,股东帐户,货币单位,备注,Unnamed: 15
1160,1488,2020-06-16,31851,nan,NaN,基金资金拨入,8175.55,8175.55,基金资金拨入,0.0,0.00,0.0,NaN,NaN,人民币,124增加金额 基金代码：001621,发?
1161,1489,2020-06-16,117149,127012,招路转债,证券买入,-8175.05,0.50,证券买入,80.0,102.18,0.0,深圳Ａ股,0152563184,人民币,证券买入,NaN
1162,1491,2020-06-17,35931,nan,NaN,基金资金拨出,-0.50,0.00,基金资金拨出,0.0,0.00,0.0,NaN,NaN,人民币,122扣除金额 基金代码：001621,发?
1163,1499,2020-06-22,232481,nan,NaN,利息归本,3.57,3.57,利息归本,0.0,0.00,0.0,NaN,NaN,人民币,利息归本: 归本利息为 3.57correc,NaN
1164,1501,2020-06-23,56524,nan,NaN,基金资金拨出,-3.57,0.00,基金资金拨出,0.0,0.00,0.0,NaN,NaN,人民币,122扣除金额 基金代码：001621,发?


# 净值数据生成

In [365]:
out_ls = [ '银行转取', '基金资金拨出',]

In [366]:
date_list = sorted(list(set(list(data['日期']))))
nav_data = pd.DataFrame(columns=['date','cash','in','out','hold_shares'])
nav_data

,date,cash,in,out,hold_shares


In [367]:
nav_data['date'] = date_list

In [368]:
nav_data['in'] = 0.
nav_data['out'] = 0.

In [369]:
for i in range(len(date_list)):
    tmp_dataframe = data[data['日期']==date_list[i]]
    nav_data.loc[i,'out'] = sum(tmp_dataframe[
        tmp_dataframe.操作.isin(out_ls)]['发生金额'])

In [370]:
nav_data[:10]

,date,cash,in,out,hold_shares
0,2019-04-09,NaN,0.0,0.00,NaN
1,2019-04-10,NaN,0.0,-70778.52,NaN
2,2019-04-11,NaN,0.0,0.00,NaN
3,2019-04-12,NaN,0.0,-5.00,NaN
4,2019-04-16,NaN,0.0,0.00,NaN
5,2019-04-17,NaN,0.0,-2864.68,NaN
6,2019-04-18,NaN,0.0,0.00,NaN
7,2019-04-19,NaN,0.0,0.00,NaN
8,2019-04-22,NaN,0.0,-1124.64,NaN
9,2019-04-23,NaN,0.0,0.00,NaN


In [279]:
tmp_dataframe = data[data['日期']==date_list[0]]

In [284]:
sum(data[data.操作.isin(out_ls)]['发生金额'])

-2067616.660000001

In [371]:
in_ls = [ '基金资金拨入','银行转存',]

In [372]:
for i in range(len(date_list)):
    tmp_dataframe = data[data['日期']==date_list[i]]
    nav_data.loc[i,'in'] = sum(tmp_dataframe[
        tmp_dataframe.操作.isin(in_ls)]['发生金额'])

In [373]:
nav_data

,date,cash,in,out,hold_shares
0,2019-04-09,NaN,70000.00,0.00,NaN
1,2019-04-10,NaN,90000.00,-70778.52,NaN
2,2019-04-11,NaN,104497.78,0.00,NaN
3,2019-04-12,NaN,0.00,-5.00,NaN
4,2019-04-16,NaN,158.21,0.00,NaN
...,...,...,...,...,...
224,2020-06-12,NaN,0.00,-1.00,NaN
225,2020-06-16,NaN,8175.55,0.00,NaN
226,2020-06-17,NaN,0.00,-0.50,NaN
227,2020-06-22,NaN,0.00,0.00,NaN


In [374]:
for i in range(len(date_list)):
    nav_data.loc[i,'cash'] = data[data['日期'] == date_list[i]].iloc[-1]['后资金额']

In [375]:
nav_data

,date,cash,in,out,hold_shares
0,2019-04-09,715.51,70000.00,0.00,NaN
1,2019-04-10,1,90000.00,-70778.52,NaN
2,2019-04-11,5,104497.78,0.00,NaN
3,2019-04-12,0,0.00,-5.00,NaN
4,2019-04-16,1,158.21,0.00,NaN
...,...,...,...,...,...
224,2020-06-12,0,0.00,-1.00,NaN
225,2020-06-16,0.5,8175.55,0.00,NaN
226,2020-06-17,0,0.00,-0.50,NaN
227,2020-06-22,3.57,0.00,0.00,NaN


In [438]:
sp_ls = list(data[data['操作']=='新股申购确认']['证券代码'])

In [493]:
data[data['证券代码']=='110066']

,index,日期,流水号,证券代码,证券名称,操作,发生金额,后资金额,摘要,成交数量,成交均价,成交金额,交易市场,股东帐户,货币单位,备注,Unnamed: 15
716,904,2020-03-05,161104,110066,盛屯发债,新股申购确认,-1000.00,1001.50,新股申购确认缴款,10.0,100.000,0.0,上海Ａ股,A400231155,人民币,债券信用配售确认,NaN
953,1207,2020-04-28,62689,110066,盛屯转债,证券买入,-21003.11,34799.42,证券买入,200.0,105.015,0.0,上海Ａ股,A400231155,人民币,证券买入,NaN
957,1211,2020-04-28,62694,110066,盛屯转债,证券卖出,21159.89,62088.46,证券卖出,-200.0,105.800,0.0,上海Ａ股,A400231155,人民币,证券卖出,NaN
1034,1292,2020-05-12,61874,110066,盛屯转债,证券卖出,10590.95,-25538.28,证券卖出,-100.0,105.910,0.0,上海Ａ股,A400231155,人民币,证券卖出,NaN
1056,1315,2020-05-14,63153,110066,盛屯转债,证券买入,-11508.16,13286.16,证券买入,110.0,104.619,0.0,上海Ａ股,A400231155,人民币,证券买入,NaN
1070,1330,2020-05-15,53100,110066,盛屯转债,证券买入,-12435.66,86974.65,证券买入,120.0,103.630,0.0,上海Ａ股,A400231155,人民币,证券买入,NaN


In [490]:
sp_ls

['754658',
 '754833',
 '370677',
 '072228',
 '072615',
 '370655',
 '733000',
 '754688',
 '370296',
 '754861',
 '072745',
 '072083',
 '733939',
 '754690',
 '754901',
 '072851',
 '370059',
 '072701',
 '733711',
 '783233',
 '733909',
 '072503',
 '787228',
 '370190',
 '787599',
 '072756']

In [457]:
match_ls = []
#bond_zh_cov_df = ak.bond_zh_cov()
#stock_em_xgsglb_df = ak.stock_em_xgsglb(market="科创板")
for i in sp_ls:
    if i not in ['787228','787599']:
        match = bond_zh_cov_df[bond_zh_cov_df['申购代码'] == i]['债券代码'].iloc[0]
        match_ls.append(match)
    else:
        match = stock_em_xgsglb_df[stock_em_xgsglb_df['申购代码']==i]['股票代码'].iloc[0]
        match_ls.append(match)

In [471]:
for i in range(len(sp_ls)):
    data.loc[data[data['证券代码']==sp_ls[i]].index,'证券代码']=match_ls[i]

In [472]:
dic ={}
origin_dic={}
i=0
flag=True
nav_data['hold_shares'] = np.nan
while i < len(date_list):
    # if i==223:
    #     print(i)
    # print(i)
    if i==0:
        flag=True
    tmp = data[data['日期'] == date_list[i]]
    for j in range(len(tmp)):
        item = tmp.iloc[j]
        if len(item['证券代码'])==6:
            dic.setdefault(item['证券代码'],0)
            dic[item['证券代码']]+=item['成交数量']
            if dic[item['证券代码']]<0:
                origin_dic.setdefault(item['证券代码'],0)
                origin_dic[item['证券代码']]+=(-(dic[item['证券代码']]))

                #origin_dic[item['证券代码']] = -(dic[item['证券代码']])
                print(str(origin_dic))
                nav_data.loc[0,'hold_shares'] = str(origin_dic)
                i=0
                dic=origin_dic
                flag=False
                break
    if flag:
        dic = {key:val for key, val in dic.items() if val != 0.}
        #print(dic)
        nav_data.loc[i,'hold_shares'] = str(dic)
        i+=1

{'113012': 100.0}
{'113012': 100.0, '123010': 90.0, '128043': 110.0, '128012': 440.0, '128015': 150.0}
{'113012': 100.0, '123010': 180.0, '128043': 220.0, '128012': 880.0, '128015': 150.0, '110066': 90.0}
{'113012': 100.0, '123010': 270.0, '128043': 330.0, '128012': 1320.0, '128015': 150.0, '110066': 90.0, '113528': 150.0}
{'113012': 100.0, '123010': 360.0, '128043': 440.0, '128012': 1760.0, '128015': 150.0, '110066': 90.0, '113528': 150.0, '128022': 200.0}


In [473]:
eval(nav_data.loc[0,'hold_shares'])

{'113012': 100.0,
 '123010': 450.0,
 '128043': 550.0,
 '128012': 2200.0,
 '128015': 150.0,
 '110066': 90.0,
 '113528': 150.0,
 '128022': 200.0}

In [474]:
nav_data

,date,cash,in,out,hold_shares
0,2019-04-09,715.51,70000.00,0.00,"{'113012': 100.0, '123010': 450.0, '128043': 5..."
1,2019-04-10,1,90000.00,-70778.52,"{'113012': 100.0, '123010': 450.0, '128043': 5..."
2,2019-04-11,5,104497.78,0.00,"{'113012': 100.0, '123010': 570.0, '128043': 5..."
3,2019-04-12,0,0.00,-5.00,"{'113012': 100.0, '123010': 570.0, '128043': 5..."
4,2019-04-16,1,158.21,0.00,"{'113012': 100.0, '123010': 570.0, '128043': 5..."
...,...,...,...,...,...
224,2020-06-12,0,0.00,-1.00,"{'123010': 360.0, '128043': 440.0, '128012': 2..."
225,2020-06-16,0.5,8175.55,0.00,"{'123010': 360.0, '128043': 440.0, '128012': 2..."
226,2020-06-17,0,0.00,-0.50,"{'123010': 360.0, '128043': 440.0, '128012': 2..."
227,2020-06-22,3.57,0.00,0.00,"{'123010': 360.0, '128043': 440.0, '128012': 2..."


## 最大回撤开始日期、结束日期、修复时间

In [60]:
roc = pd.read_excel('../dataset/rtn.xlsx')

In [66]:
roc['nav'] = np.nan

In [69]:
roc.loc[0,'nav'] = 1+roc.loc[0,'rtn']

In [71]:
for i in range(1,len(roc)):
    roc.loc[i,'nav'] = roc.loc[i-1,'nav']*(1+roc.loc[i,'rtn'])

In [72]:
roc

,date,rtn,nav
0,2020-06-19,0.007144,1.007144
1,2020-06-22,0.002819,1.009983
2,2020-06-23,0.000002,1.009986
3,2020-06-24,0.000002,1.009988
4,2020-06-29,0.000002,1.009991
...,...,...,...
83,2020-10-26,0.000190,1.084437
84,2020-10-27,-0.000422,1.083980
85,2020-10-28,-0.004971,1.078591
86,2020-10-29,-0.005522,1.072635


In [91]:
roc.loc[roc[roc.nav==max(roc['nav'][:3])].index,'date']

2   2020-06-23
Name: date, dtype: datetime64[ns]

In [113]:
start = None
end = None
max_drawdown = 0
for i in range(1,len(roc)):
    tmp = (max(roc['nav'][:i])-roc.loc[i,'nav'])/max(roc['nav'][:i])
    if tmp>=max_drawdown:
        max_drawdown = tmp
        start = roc.iloc[roc[roc.nav == max(roc['nav'][:i])].index]['date']
        start = start.iloc[0]
        end = roc.iloc[i]['date']
print('start:',start)
print('end:',end)
print('max_drawdown:',max_drawdown)

start: 2020-08-27 00:00:00
end: 2020-10-29 00:00:00
max_drawdown: 0.05562003730207254


In [140]:
# 求修复时间，从最大回撤后点开始计算
recovery = np.nan
for i in range(roc[roc['date']==end].index[0],len(roc)):
    if roc.iloc[i]['nav']>=roc.iloc[roc[roc['date']==start].index[0]]['nav']:
        recovery = roc.iloc[i]['date']
        break
recovery

nan

# akshare获取仓位单价数据

In [10]:
import akshare as ak

In [415]:
sp_ls = list(data[data['操作']=='新股申购确认']['证券代码'])

In [534]:
xgsg_ls = []
for i in sp_ls:
    if len(bond_zh_cov_df[bond_zh_cov_df['申购代码'] == i])==0:
        xgsg_ls.append(i)

In [537]:
xgsg_match_ls=[]
for i in xgsg_ls:
    xgsg_match_ls.append(stock_em_xgsglb_df[stock_em_xgsglb_df['申购代码']==i]['股票代码'].iloc[0])
xgsg_match_ls


['688228', '688599']

In [579]:
xgsg_match_ls

['688228', '688599']

In [585]:
len(nav_data)

229

In [584]:
len(nav_ls)

131

In [618]:
nav_ls=[]
for j in range(len(nav_data)):
    tmp_hs = eval(nav_data.loc[j,'hold_shares'])
    tmp_date = nav_data.loc[j,'date']
    sum0=0
    print(tmp_date)
    for i,v in tmp_hs.items():
        if i in xgsg_match_ls:
            if i[:2] in ['11','68','60']:
                stock_name = 'sh'+i
            if i[:2] in ['12','00']:
                stock_name='sz'+i
            bond_zh_hs_cov_daily_df = ak.bond_zh_hs_cov_daily(symbol=stock_name)
            if tmp_date>=bond_zh_hs_cov_daily_df.index[0]:
                unit_price = bond_zh_hs_cov_daily_df.loc[bond_zh_hs_cov_daily_df.index<=tmp_date,'close'].iloc[-1]
            else:
                unit_price = stock_em_xgsglb_df[stock_em_xgsglb_df['股票代码']==i]['发行价格'].iloc[0]
            sum0+=v*unit_price
        else:
            if i[:2] in ['11','68','60','51']:
                stock_name = 'sh'+i
            if i[:2] in ['12','00']:
                stock_name='sz'+i
            bond_zh_hs_cov_daily_df = ak.bond_zh_hs_cov_daily(symbol=stock_name)
            if tmp_date>=bond_zh_hs_cov_daily_df.index[0]:
                unit_price = bond_zh_hs_cov_daily_df.loc[bond_zh_hs_cov_daily_df.index<=tmp_date,'close'].iloc[-1]
            else:
                unit_price = bond_zh_cov_df[bond_zh_cov_df['债券代码'] == i]['发行价格'].iloc[0]
            sum0+=v*unit_price
    nav_ls.append(sum0)
            
        

2019-04-09 00:00:00
2019-04-10 00:00:00
2019-04-11 00:00:00
2019-04-12 00:00:00
2019-04-16 00:00:00
2019-04-17 00:00:00
2019-04-18 00:00:00
2019-04-19 00:00:00
2019-04-22 00:00:00
2019-04-23 00:00:00
2019-04-24 00:00:00
2019-04-26 00:00:00
2019-04-29 00:00:00
2019-05-07 00:00:00
2019-05-08 00:00:00
2019-05-09 00:00:00
2019-06-06 00:00:00
2019-06-10 00:00:00
2019-06-12 00:00:00
2019-06-13 00:00:00
2019-06-14 00:00:00
2019-06-17 00:00:00
2019-06-18 00:00:00
2019-06-19 00:00:00
2019-06-20 00:00:00
2019-06-21 00:00:00
2019-06-24 00:00:00
2019-06-25 00:00:00
2019-06-26 00:00:00
2019-06-28 00:00:00
2019-07-01 00:00:00
2019-07-02 00:00:00
2019-07-03 00:00:00
2019-07-04 00:00:00
2019-07-05 00:00:00
2019-07-08 00:00:00
2019-07-09 00:00:00
2019-07-12 00:00:00
2019-07-15 00:00:00
2019-07-17 00:00:00
2019-07-18 00:00:00
2019-07-22 00:00:00
2019-07-23 00:00:00
2019-07-24 00:00:00
2019-07-25 00:00:00
2019-07-26 00:00:00
2019-07-29 00:00:00
2019-07-30 00:00:00
2019-08-01 00:00:00
2019-08-02 00:00:00


In [620]:
nav_data['hold_shares_value'] = pd.Series(nav_ls)

In [622]:
nav_data.to_csv('/Users/lanruitong/Desktop/nav_data.csv')

In [2]:
nav_data = pd.read_csv('/Users/lanruitong/Desktop/nav_data.csv')

In [6]:
nav_data['rtn'] = np.nan

In [10]:
nav_data.head(3)

,Unnamed: 0,date,cash,in,out,hold_shares,hold_shares_value,rtn
0,0,2019-04-09,715.51,70000.00,0.00,"{'113012': 100.0, '123010': 450.0, '128043': 5...",419632.80,NaN
1,1,2019-04-10,1.00,90000.00,-70778.52,"{'113012': 100.0, '123010': 450.0, '128043': 5...",436520.64,NaN
2,2,2019-04-11,5.00,104497.78,0.00,"{'113012': 100.0, '123010': 570.0, '128043': 5...",532952.90,NaN


In [11]:
for i in range(1,len(nav_data)):
    value1 = nav_data.loc[i,'cash']+nav_data.loc[i,'hold_shares_value']-(nav_data.loc[i,'in']+nav_data.loc[i,'out'])
    value0 = nav_data.loc[i-1,'cash']+nav_data.loc[i-1,'hold_shares_value']
    nav_data.loc[i,'rtn'] = (value1-value0)/value0

In [14]:
nav_data.to_csv('/Users/lanruitong/Desktop/nav_data.csv')

In [588]:
tmp_hs = eval(nav_data.loc[179,'hold_shares'])
tmp_date = nav_data.loc[179,'date']

In [589]:
tmp_date

Timestamp('2020-03-18 00:00:00')

In [616]:
bond_zh_hs_cov_daily_df = ak.bond_zh_hs_cov_daily(symbol='sh510050')
bond_zh_hs_cov_daily_df


,open,high,low,close,volume
date,,,,,
2005-02-23,0.881,0.882,0.866,0.876,1.269743e+09
2005-02-24,0.876,0.876,0.868,0.876,4.516142e+08
2005-02-25,0.877,0.887,0.875,0.880,5.064607e+08
2005-02-28,0.878,0.879,0.870,0.872,1.879652e+08
2005-03-01,0.870,0.873,0.865,0.867,2.080945e+08
...,...,...,...,...,...
2020-11-27,3.498,3.561,3.495,3.557,8.436767e+08
2020-11-30,3.516,3.575,3.472,3.473,1.467465e+09
2020-12-01,3.476,3.563,3.476,3.557,8.116321e+08


In [590]:
tmp_hs

{'113012': 100.0,
 '123010': 360.0,
 '128043': 440.0,
 '128012': 2680.0,
 '128015': 150.0,
 '110066': 100.0,
 '113528': 150.0,
 '128022': 400.0,
 '127003': 628.0,
 '128062': 230.0,
 '601668': 18900.0,
 '128013': 300.0,
 '601166': 3900.0,
 '601288': 10000.0,
 '600028': 3500.0,
 '113032': 10.0,
 '110033': 660.0,
 '600887': 500.0,
 '510300': 10000.0,
 '110067': 10.0,
 '128100': 10.0,
 '510050': 11000.0}

In [609]:
unit_price = bond_zh_cov_df[bond_zh_cov_df['债券代码'] == '601668']['发行价格'].iloc[0]
unit_price

IndexError: single positional indexer is out-of-bounds

In [ ]:
bond_zh_hs_cov_daily_df = ak.bond_zh_hs_cov_daily(symbol='sz510050')


In [575]:
bond_zh_hs_cov_daily_df.index

DatetimeIndex(['2018-08-27', '2018-08-28', '2018-08-29', '2018-08-30',
               '2018-08-31', '2018-09-03', '2018-09-04', '2018-09-05',
               '2018-09-06', '2018-09-07',
               ...
               '2020-09-25', '2020-09-28', '2020-09-29', '2020-09-30',
               '2020-10-09', '2020-10-12', '2020-10-13', '2020-10-14',
               '2020-10-15', '2020-10-16'],
              dtype='datetime64[ns]', name='date', length=506, freq=None)

In [577]:
bond_zh_hs_cov_daily_df.loc[bond_zh_hs_cov_daily_df.index<=tmp_date,'close'].iloc[-1]

114.09

In [572]:
bond_zh_hs_cov_daily_df[
    (bond_zh_hs_cov_daily_df.index>=pd.to_datetime('20190407'))&
    (bond_zh_hs_cov_daily_df.index<=pd.to_datetime('20190520'))]

,open,high,low,close,volume
date,,,,,
2019-04-08,116.720,117.738,115.600,116.200,77561.0
2019-04-09,116.200,116.200,115.010,115.181,31768.0
2019-04-10,115.000,115.000,113.403,114.090,42703.0
2019-04-25,125.499,198.000,125.499,171.811,1782377.0
2019-04-26,179.990,198.899,172.651,195.900,2057370.0
2019-04-29,198.590,208.650,182.000,186.150,2361170.0
2019-04-30,193.000,199.990,186.151,199.990,2229262.0
2019-05-06,205.000,215.400,197.000,212.022,3625912.0
2019-05-07,211.500,213.666,165.150,180.011,7224114.0


In [556]:
bond_zh_hs_cov_daily_df.loc[pd.to_datetime('20190410')]['close']

114.09

In [529]:
stock_em_xgsglb_df[stock_em_xgsglb_df['股票代码']=='688228']['发行价格'].iloc[0]


59.26

In [544]:
sum0 = 0
for i,v in tmp_hs.items():
    if i[:2] in ['11','68','60']:
        stock_name = 'sh'+i
    if i[:2] in ['12','00']:
        stock_name='sz'+i
    bond_zh_hs_cov_daily_df = ak.bond_zh_hs_cov_daily(symbol=stock_name)
    if tmp_date>=bond_zh_hs_cov_daily_df.index[0]:
        unit_price = bond_zh_hs_cov_daily_df.loc[tmp_date,'close']
    else:
        unit_price = bond_zh_cov_df[bond_zh_cov_df['债券代码'] == i]['发行价格'].iloc[0]
    sum0+=v*unit_price
sum0

KeyError: Timestamp('2019-04-11 00:00:00')

In [59]:
data[data['证券代码']=='791077']

,日期,流水号,证券代码,证券名称,操作,发生金额,后资金额,摘要,成交数量,成交均价,成交金额,交易市场,股东帐户,货币单位,备注,Unnamed: 15
390,2019-10-14,100373113,791077,渝农配号,申购配号,0.0,7.0,申购配号,1.0,0.0,0.0,上海Ａ股,A400231155,人民币,起始配号:100023554797,NaN


In [613]:
data[data['证券代码']=='510300']

,index,日期,流水号,证券代码,证券名称,操作,发生金额,后资金额,摘要,成交数量,成交均价,成交金额,交易市场,股东帐户,货币单位,备注,Unnamed: 15
742,944,2020-03-13,82407,510300,300ETF,证券买入,-37660.71,51999.19,证券买入,10000.0,3.765,0.0,上海Ａ股,A400231155,人民币,证券买入,NaN
828,1045,2020-03-20,66174,510300,300ETF,证券卖出,35931.01,-77893.80,证券卖出,-10000.0,3.594,0.0,上海Ａ股,A400231155,人民币,证券卖出,NaN


In [519]:
bond_zh_hs_cov_daily_df = ak.bond_zh_hs_cov_daily(symbol="sz002310")
print(bond_zh_hs_cov_daily_df)

              open    high     low   close      volume
date                                                  
2009-11-27   99.00  118.50   99.00  116.50  10104167.0
2009-11-30  123.00  127.60  118.00  124.00   4920957.0
2009-12-01  121.35  124.78  119.20  123.78   2488330.0
2009-12-02  127.00  130.44  120.51  122.00   3145790.0
2009-12-03  121.00  125.49  120.88  125.05   2514623.0
...            ...     ...     ...     ...         ...
2020-11-26    4.74    4.77    4.72    4.73  13738109.0
2020-11-27    4.75    4.80    4.72    4.76  16241634.0
2020-11-30    4.76    4.80    4.76    4.80  20212757.0
2020-12-01    4.80    4.82    4.76    4.81  14389150.0
2020-12-02    4.81    4.81    4.78    4.80  12331686.0

[2445 rows x 5 columns]


In [427]:
bond_zh_hs_cov_daily_df.loc[pd.to_datetime('20200610'),'close']

17.21

In [45]:
bond_zh_cov_df = ak.bond_zh_cov()


In [498]:
bond_zh_cov_df[bond_zh_cov_df['债券代码'] == '110066']

,债券代码,交易场所,债券简称,申购日期,申购代码,正股代码,正股简称,债券面值,发行价格,转股价,...,备忘录,正股价,市场类型,股权登记日,申购上限,转股价值,债现价,转股溢价率,每股配售额,发行规模
158,110066,CNSESH,盛屯转债,2020-03-02T00:00:00,733711,600711,盛屯矿业,100.0,100.0,4.92,...,"本次发行的可转债向发行人在股权登记日(2020年2月28日,T-1日)收市后中国结算上海分公...",6.11,1,2020-02-28T00:00:00,100,125.2049,134,7.02,1.033,23.86456


In [317]:
stock_em_xgsglb_df = ak.stock_em_xgsglb(market="科创板")


In [531]:
stock_em_xgsglb_df[stock_em_xgsglb_df['申购代码']=='787599']['股票代码'].iloc[0]

'688599'

In [436]:
data[data['操作']=='新股申购确认']

,index,日期,流水号,证券代码,证券名称,操作,发生金额,后资金额,摘要,成交数量,成交均价,成交金额,交易市场,股东帐户,货币单位,备注,Unnamed: 15
121,129,2019-07-03,80405,754658,安图发债,新股申购确认,-1000.0,5.00,新股申购确认缴款,10.0,100.00,0.0,上海Ａ股,A400231155,人民币,债券信用配售确认,NaN
228,257,2019-08-21,74139,754833,欧派发债,新股申购确认,-1000.0,3.50,新股申购确认缴款,10.0,100.00,0.0,上海Ａ股,A400231155,人民币,债券信用配售确认,NaN
229,262,2019-08-21,79202,370677,英科发债,新股申购确认,-1000.0,1003.50,新股申购确认缴款,10.0,100.00,0.0,深圳Ａ股,0152563184,人民币,债券信用配售确认,NaN
230,263,2019-08-21,79205,072228,合兴发债,新股申购确认,-1000.0,3.50,新股申购确认缴款,10.0,100.00,0.0,深圳Ａ股,0152563184,人民币,债券信用配售确认,NaN
249,287,2019-08-27,82029,072615,哈尔发债,新股申购确认,-1000.0,3.00,新股申购确认缴款,10.0,100.00,0.0,深圳Ａ股,0152563184,人民币,债券信用配售确认,NaN
264,309,2019-09-03,83927,370655,晶瑞发债,新股申购确认,-2000.0,15442.74,新股申购确认缴款,20.0,100.00,0.0,深圳Ａ股,0152563184,人民币,债券信用配售确认,NaN
361,434,2019-10-31,80434,733000,浦发发债,新股申购确认,-3000.0,1001.50,新股申购确认缴款,30.0,100.00,0.0,上海Ａ股,A400231155,人民币,债券信用配售确认,NaN
362,435,2019-10-31,80436,754688,石英发债,新股申购确认,-1000.0,1.50,新股申购确认缴款,10.0,100.00,0.0,上海Ａ股,A400231155,人民币,债券信用配售确认,NaN
384,471,2019-11-19,80169,370296,利德发债,新股申购确认,-1000.0,1000.00,新股申购确认缴款,10.0,100.00,0.0,深圳Ａ股,0152563184,人民币,债券信用配售确认,NaN
385,475,2019-11-20,75978,754861,白电发债,新股申购确认,-1000.0,0.00,新股申购确认缴款,10.0,100.00,0.0,上海Ａ股,A400231155,人民币,债券信用配售确认,NaN


In [409]:
tmp=nav_data[(nav_data['date']>=pd.to_datetime('20200327'))]

In [411]:
tmp.iloc[0]['hold_shares']

"{'123010': 2520.0, '128043': 3080.0, '128012': 12820.0, '110067': 10.0, '128100': 10.0, '113012': 100.0, '128015': 150.0, '110066': 100.0, '123049': 10.0, '113528': 350.0, '128022': 450.0, '688599': 500.0, '754658': 10.0, '754833': 10.0, '370677': 10.0, '072228': 10.0, '072615': 10.0, '370655': 20.0, '601668': 19100.0, '733000': 30.0, '754688': 10.0, '370296': 10.0, '754861': 10.0, '072745': 10.0, '072083': 20.0, '733939': 20.0, '754690': 10.0, '754901': 10.0, '072851': 10.0, '370059': 10.0, '072701': 10.0, '601166': 3900.0, '601288': 10000.0, '600028': 3500.0, '733711': 10.0, '783233': 10.0, '733909': 10.0, '072503': 10.0, '787228': 500.0, '113008': 500.0, '113016': 200.0, '110033': 250.0, '127003': 500.0, '128062': 200.0, '123019': 300.0}"